# Carregando Pacotes e Ajustando a GPU

In [3]:
from transformers import pipeline, AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
import pdfplumber

In [4]:
device = 0 if torch.cuda.is_available() else -1
print(f"🔧 Dispositivo: {'GPU' if device == 0 else 'CPU'}")

🔧 Dispositivo: GPU


# Variaveis Globais .

In [5]:
MODELO_EXTRATIVO = "pierreguillou/bert-large-cased-squad-v1.1-portuguese"
MODELO_SEMANTICO = "neuralmind/bert-base-portuguese-cased"
MODELO_GERATIVO = "facebook/bart-base"  # Modelo leve e estável

In [6]:
# 3. Carregamento à Prova de Erros
try:
    print("⏳ Carregando modelos...")
    
    # Extrativo (seu modelo original)
    qa_pipeline = pipeline(
        "question-answering",
        model=MODELO_EXTRATIVO,
        tokenizer=MODELO_EXTRATIVO,
        device=device
    )
    
    # Semântico (para busca)
    model_embed = SentenceTransformer(MODELO_SEMANTICO)
    
    # Generativo (alternativa estável)
    tokenizer_gen = AutoTokenizer.from_pretrained(MODELO_GERATIVO)
    model_gen = AutoModel.from_pretrained(MODELO_GERATIVO).to(device)
    
    print("✅ Modelos carregados com sucesso!")
except Exception as e:
    print(f"❌ Falha crítica: {str(e)}")
    raise

⏳ Carregando modelos...


Device set to use cuda:0
No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


✅ Modelos carregados com sucesso!


# Funções

In [7]:
def carregar_documento(caminho):
    """Mantido igual ao seu original"""
    with pdfplumber.open(caminho) as pdf:
        return " ".join(page.extract_text() for page in pdf.pages if page.extract_text())

In [8]:
def dividir_texto(texto, tamanho=400, sobreposicao=100):
    """Versão otimizada da sua função"""
    palavras = texto.split()
    return [
        " ".join(palavras[i:i + tamanho])
        for i in range(0, len(palavras), tamanho - sobreposicao)
    ]

In [9]:
def buscar_trechos(pergunta, trechos, top_k=3):
    """Busca semântica simplificada"""
    emb_pergunta = model_embed.encode(pergunta)
    emb_trechos = model_embed.encode(trechos)
    simil = cosine_similarity([emb_pergunta], emb_trechos)[0]
    return [trechos[i] for i in np.argsort(simil)[-top_k:][::-1]]

In [10]:
def responder(pergunta, contexto):
    """Resposta direta sem gerador complexo"""
    resultado = qa_pipeline(question=pergunta, context=contexto)
    return {
        "resposta": resultado["answer"],
        "score": resultado["score"],
        "contexto": contexto[:200] + "..."  # Preview
    }

# Processando

In [11]:
print("\n📄 Processando documento...")
texto = carregar_documento("cabelos cacheados.pdf")
trechos = dividir_texto(texto)


📄 Processando documento...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [15]:
pergunta = "Quantos são os tipos de cabelos cacheados?"

In [16]:
relevantes = buscar_trechos(pergunta, trechos)
melhor = responder(pergunta, " ".join(relevantes))

In [17]:
print(f"\n💡 Resposta (score: {melhor['score']:.2f}):")
print(melhor["resposta"])
print(f"\n📌 Contexto: {melhor['contexto']}\n")
        


💡 Resposta (score: 0.43):
Tipo 2 (ondulado), Tipo 3

📌 Contexto: Definição e Tipos de Cachos Cabelos cacheados são caracterizados por sua estrutura em espiral ou ondulada, resultante de folículos pilosos assimétricos. Essa forma se deve à distribuição desigual de q...

